# Geographically specific LCI and mapping with GLAM and Impact World + factors

Useful resources: <br>
https://stackoverflow.com/questions/72115909/get-all-elementary-flows-generated-by-an-activity-in-brightway <br>
https://stackoverflow.com/questions/75914149/unable-to-use-bw2calc-lca-to-dafaframe-to-export-lci-data <br>
https://stackoverflow.com/questions/77351101/named-life-cycle-inventory <br>

## Setup

In [ ]:
# If you do not have the following libraries installed, you can install them using the 'pip install' command (for instance, 'pip install pandas')
import bw2io as bi 
import bw2data as bd
import bw2calc as bc
import pandas as pd
import os
from functools import lru_cache
from IPython.display import clear_output

In [ ]:
bd.projects.set_current('Extracting_LCI') # activates a project, or creates it first if it doesn't exist yet

In [ ]:
# Run this if you haven't imported ecoinvent yet:
#bi.import_ecoinvent_release(
#        version='3.10.1',  # or whichever version you want to use
#        system_model='cutoff', # can be cutoff / apos / consequential / EN15804
#        username='yourEcoinventUsername',
#        password='yourEcoinventPassword'
#    )

In [ ]:
ecoinvent=bd.Database('ecoinvent-3.10.1-cutoff')

## Brightway examples

This section is just providing examples of how to find a particular activity or LCIA method in the ecoinvent database, and how to perform an LCA.

### Activity selection

In [ ]:
prod_name='sawnwood, beam, softwood, dried (u=10%), planed'
act_name='market for '+prod_name
location='Europe without Switzerland'
unit='cubic meter'

In [ ]:
def select_activity(act_name, prod_name, location, unit):
    act_list = [x for x in ecoinvent if act_name.lower() == x['name'].lower() and prod_name.lower() == x['reference product'].lower() and location.lower() == x['location'].lower() and unit.lower() == x['unit'].lower()]
    assert len(act_list)==1, 'There should be exactly one activity matching the criteria, we found '+str(len(act_list))+': '+str(act_list)
    return act_list[0]

In [ ]:
activity=select_activity(act_name, prod_name, location, unit)
print(activity)

### Method selection

In [ ]:
method=('IMPACT World+ v2.1, footprint version',  'ecosystem quality',  'remaining ecosystem quality damage')
# Note that the footprint and expert versions are quite different! https://zenodo.org/records/14041258

In [ ]:
def select_method(method):
    method_list=[x for x in bd.methods if str(method) in str(x)]
    assert len(method_list)==1, 'There should be exactly one method matching the criteria, we found '+str(len(method_list))+': '+str(method_list)
    return method_list[0]

In [ ]:
IWplus=select_method(method)

### LCA calculation

In [ ]:
lca=bc.LCA({activity: 1}, bd.Method(IWplus).name)

In [ ]:
lca.lci()
lca.lcia()

In [ ]:
lca.score

## Exporting regionalized LCI

Next, we'll look at how to extract the regionalized inventory on a single case (the example above).

First, we define a simple function to look up activities in ecoinvent in an efficient way.

In [ ]:
@lru_cache(maxsize=16000) # lru_cache speeds up repeated lookups. Do not omit it or you might run out of memory and crash.
def cached_lookup(key):
    return bd.get_activity(key)

To export the regionalized inventory, we need to check each possible combination of elementary flow and activity in our LCA. If the amount of a specific elementary flow for a specific activity is not zero, then we look up information for this activity in the ecoinvent database, such as name, category, unit, reference product, and, in particular: location.

In [ ]:
def regio_inventory(lca):
    inventory_results = {'amount':[], 'elementary_flow':[], 'EF_compartment':[], 'EF_unit':[], 'activity_name':[], 'activity_location':[], 'reference_product':[]}
    maxrow=len(lca.biosphere_dict)-1

    for row_key, row in lca.biosphere_dict.items(): # iterate over elementary flows
        clear_output(wait=True)
        print('reading elementary flow number ',row, 'of ', maxrow)
        for col_key, col in lca.activity_dict.items(): # iterate over activities
            if lca.inventory[row, col] != 0: # check in the full inventory matrix if the value for this elementary flow and activity is not zero
                ef=cached_lookup(row_key) # look up the elementary flow 
                act=cached_lookup(col_key) # look up the activity
                inventory_results['amount'].append(lca.inventory[row, col]) 
                inventory_results['elementary_flow'].append(ef.get('name'))
                inventory_results['EF_compartment'].append(ef.get('categories'))
                inventory_results['EF_unit'].append(ef.get('unit'))
                inventory_results['activity_name'].append(act.get('name'))
                inventory_results['activity_location'].append(act.get('location'))
                inventory_results['reference_product'].append(act.get('reference product'))
                
    return inventory_results

Let's try it on our example LCA - careful, this takes close to an hour to run on my computer.

In [ ]:
inv=regio_inventory(lca)

Right now the result is a dictionary. To export this as a table, readable in Excel, we convert it to a DataFrame:

In [ ]:
inv_df=pd.DataFrame.from_dict(inv)
inv_df.to_excel("your-path-and-file-name.xlsx")
# To read it again: 
# inv_df=pd.read_excel("your-path-and-file-name.xlsx", sheet_name='Sheet1')

## Automating it

First, let's import the list of all products we want to treat.

In [ ]:
product_list=pd.read_excel("path_to_Example product list.xlsx")

In [ ]:
product_list

Let's make a list with all the market activities.

In [ ]:
activity_list=[]
for i in product_list.index:
    print('activity: '+product_list.loc[i,'Ecoinvent process']+' _ product: '+product_list.loc[i,'Product name']+' _ geography: '+product_list.loc[i,'Geography']+' _ unit: '+product_list.loc[i,'Ecoinvent unit'])
    act=select_activity(product_list.loc[i,'Ecoinvent process'], product_list.loc[i,'Product name'], product_list.loc[i,'Geography'], product_list.loc[i,'Ecoinvent unit'])
    activity_list.append(act)

Define a folder where you will store your inventories:

In [ ]:
prod_path="your-path-to-an-inventory-folder-ending-with\\"


In [ ]:
already_done_files=os.listdir(prod_path) # In case you run this code multiple times, you can skip already done products.
already_done_prod=[]
for a in already_done_files:
    b=a.replace('.xlsx','')
    already_done_prod.append(b.split('__')) # I use a double underscore to separate the product name, the geography and the unit in the file name

As before, we run the function to regionalize the inventory, but this time we run it on the full list of activity and automatically save each inventory in the folder. Note that this might take hours, if not days, depending on the size of your list of products.

In [ ]:
for a in activity_list:
    if [a['reference product'],a['location'],a['unit']] not in already_done_prod:
        lca=bc.LCA({a: 1})
        lca.lci()
        
        reg_inv=regio_inventory(lca)
        reg_inv_df=pd.DataFrame.from_dict(reg_inv)
        
        reg_inv_df.to_excel(prod_path+a['reference product'].replace('/','--')+'__'+a['location']+'__'+a['unit']+".xlsx") # If there's a slash in the product name, it will not work, so I replace it with a double dash.

## Mapping with GLAM factors

Now that we have regionalized inventories, let's look at how we can connect them with the right characterization factors from GLAM and Impact World +

First, we need to import appropriate mapping tables, matching the name of flows in GLAM or Impact World + to ecoinvent.

In [ ]:
mapping_table_GLAM=pd.read_excel("your-path-to-mappingTableGLAM.xlsx") # Mapping for GLAM land use flows - note that it is homemade, and might not reflect "official" mapping tables developed at a later point.

In [ ]:
mapping_table_IW=pd.read_excel("your-path-to-mappingTableIWPlus.xlsx") # Mapping table for Impact World+ flows 

In [ ]:
geo_mapping=pd.read_excel("your-path-to-mappingTableGeographies.xlsx") # Mapping for geographies - here too, the mapping is homemade and some liberties have been taken with approximating larger regions as single countries (e.g. EU as Germany) but this should only affect very few flows in ecoinvent.
# See https://support.ecoinvent.org/geographies for more information.

We also import the GLAM characterization factors:

In [ ]:
GLAM_landuse_path="your-path-to-GLAM_template_EQ_Land_Use.xlsx"  # You need to download the GLAM characterization factors from https://www.lifecycleinitiative.org/activities/life-cycle-assessment-data-and-methods/global-guidance-for-life-cycle-impact-assessment-indicators-and-methods-glam/ 
GLAM_df=pd.read_excel(GLAM_landuse_path,sheet_name='lciamethods_CF')
GLAM_land = GLAM_df[(GLAM_df['FLOW_class0']=='Land use') & (GLAM_df['Species']=='Aggregated') & (GLAM_df['LCIAMethod_type']=='Damage level') & (GLAM_df['LCIAMethod_name']=='EQ Land use') & (GLAM_df['Scenario']!='original_weighting_ecoregion_area') & (GLAM_df['Scenario']!='proxy_weighting_ecoregion_area') & (GLAM_df['LCIAMethod_location_name']!='West Bank')]
# We select only :
# - the aggregated damage on all species, not for each species
# - the damage level (not the midpoint level)
# - the original_weighting_ecoregion_area scenario
# - somehow there is a geographic code that corresponds both to West Bank and to Gaza. I only kepy Gaza to avoid double-counting (this is unlikely to influence the results).
GLAM_land.loc[:,'RegionalOrGlobal']=GLAM_land.loc[:,'Unit'].str.split(n=1, expand=True)[0] # The info on whether the CF is regional or global is in the unit, so we split it and keep only the first part


We reformat them using a pivot table (this will come in handy later):

In [ ]:
GLAM_land_pivot=GLAM_land.pivot(index=['FLOW_name_org','FLOW_class1','LCIAMethod_location'], columns=['LCIAMethod_mathematicalApproach','RegionalOrGlobal'], values='CF')
GLAM_land_pivot.columns=['Average Regional CF','Average Global CF','Marginal Regional CF','Marginal Global CF']

Similarly, we can import Impact World +

In [ ]:
iwplus=pd.read_excel("your-path-to.impact_world_plus_2.1_footprint_version_ecoinvent_v311.xlsx") # Or your preferred version of Impact World + Downloaded from https://zenodo.org/records/14041258
iwplus_land=iwplus[(iwplus['Impact category'].str.contains('Land ')) & (iwplus['MP or Damage'].str.contains('Damage'))]

Next, we can define a function to match each item in an inventory to the appropriate characterization factors.

In [ ]:
def mapper_land(inv_df, GLAM_land_pivot, mapping_table_GLAM, mapping_table_IW, geo_mapping, iwplus_land): 
 
    # Map the ecoinvent inventory data to GLAM flow names
    inv_df_mapped=pd.merge(inv_df, mapping_table_GLAM[['GLAM flow','Ecoinvent','Type','Type2']], how='left', left_on='elementary_flow', right_on='Ecoinvent')
    inv_df_mapped=inv_df_mapped.drop(columns=['Ecoinvent'])

    # Map to geography codes
    inv_df_geo=pd.merge(inv_df_mapped, geo_mapping[['Shortname','GLAM_geo_code','IW+_geo_code']], how='left', left_on='activity_location', right_on='Shortname')
    inv_df_geo=inv_df_geo.drop(columns=['Shortname'])

    # Map to GLAM land use characterization factors
    inv_df_CFs_GLAM=pd.merge(inv_df_geo, GLAM_land_pivot, how='left', left_on=['GLAM flow','Type2', 'GLAM_geo_code'], right_index=True)

    # Map to IW+ flows
    inv_df_CFs_GLAM_IWmap=pd.merge(inv_df_CFs_GLAM, mapping_table_IW[['IW+','ecoinvent']], how='left', left_on='elementary_flow', right_on='ecoinvent')
    inv_df_CFs_GLAM_IWmap=inv_df_CFs_GLAM_IWmap.drop(columns=['ecoinvent'])
    inv_df_CFs_GLAM_IWmap['IW+_Flow']=inv_df_CFs_GLAM_IWmap['IW+']+', '+inv_df_CFs_GLAM_IWmap['IW+_geo_code'] # Build the full name of the IW+ flow by combining the IW+ flow name and the geography code

    # Map to IW+ characterization factors
    inv_df_CFs_iw=pd.merge(inv_df_CFs_GLAM_IWmap, iwplus_land[['Elem flow name', 'CF value']], how='left', left_on='IW+_Flow', right_on='Elem flow name')
    inv_df_CFs_iw=inv_df_CFs_iw.drop(columns=['Elem flow name'])
    inv_df_CFs_iw=inv_df_CFs_iw.rename(columns={'CF value':'CF_IW+'})

    for i in inv_df_CFs_iw.index: # The name of the GLAM flows does not indicate whether they are transformations "to" or "from" a specific land use type, so we need to check the Type column and invert the CFs if necessary.
        if str(inv_df_CFs_iw.loc[i,'Type'])=='from':
            inv_df_CFs_iw.loc[i,'Average Regional CF']=-inv_df_CFs_iw.loc[i,'Average Regional CF']
            inv_df_CFs_iw.loc[i,'Average Global CF']=-inv_df_CFs_iw.loc[i,'Average Global CF']
            inv_df_CFs_iw.loc[i,'Marginal Regional CF']=-inv_df_CFs_iw.loc[i,'Marginal Regional CF']
            inv_df_CFs_iw.loc[i,'Marginal Global CF']=-inv_df_CFs_iw.loc[i,'Marginal Global CF']

    inv_df_impact=inv_df_CFs_iw
    inv_df_impact['Average Regional Impact']=inv_df_impact['amount']*inv_df_impact['Average Regional CF']
    inv_df_impact['Average Global Impact']=inv_df_impact['amount']*inv_df_impact['Average Global CF']
    inv_df_impact['Marginal Regional Impact']=inv_df_impact['amount']*inv_df_impact['Marginal Regional CF']
    inv_df_impact['Marginal Global Impact']=inv_df_impact['amount']*inv_df_impact['Marginal Global CF']
    inv_df_impact['Impact_IW+']=inv_df_impact['amount']*inv_df_impact['CF_IW+']
    
    return inv_df_impact

Then we iterate for all products


In [ ]:
inventory_files=os.listdir(prod_path)
characterized_inventories="your-path-to-a-folder-ending-with\\"
all_results=[]
for prod in inventory_files:
    # This part will create a characterized inventory and save it in the characterized_inventories folder.
    inv_df=pd.read_excel(prod_path+'\\'+prod)
    characterized_inventory=mapper_land(inv_df, GLAM_land_pivot, mapping_table_GLAM, mapping_table_IW, geo_mapping, iwplus_land)
    characterized_inventory[characterized_inventory['EF_compartment'].str.contains('land')].to_excel(characterized_inventories+prod)

    # This part will gather all results in a single table:
    results=characterized_inventory[characterized_inventory['EF_compartment'].str.contains('land')][['Average Regional Impact','Average Global Impact','Marginal Regional Impact','Marginal Global Impact', 'Impact_IW+']].sum()
    results_df=pd.DataFrame(results, columns=[prod.replace('.xlsx','')])
    all_results.append(results_df)

all_results_df=pd.concat(all_results, axis=1).T

In [ ]:
all_results_df

## Changing geography and forestry type

The following code will change the location of all activities for the main product in the inventory ONLY:

In [ ]:
def geo_remapper(inventory_df,country_code):
    inv_df_sort=inventory_df[inventory_df['EF_compartment'].str.contains('land')].sort_values(by=['amount'], ascending=False)
    main_product=inv_df_sort.iloc[0]['reference_product']
    for i in inv_df_sort.index:
        if inv_df_sort.loc[i,'reference_product']==main_product:
            inv_df_sort.loc[i,'activity_location']=country_code
    return inv_df_sort

In [ ]:
new_countries=['DE', 'SE', 'FI', 'NO', 'DK', 'PL', 'EE', 'LV'] # Make a list of country codes to remap the geography of the inventory data

In [ ]:
path_geo_inventories="your_path_to_inventories_with_remapped_geographies\\"

In [ ]:
inventory_files=os.listdir(prod_path)
for prod_name in inventory_files:
    [activity, geography, unit]=prod_name.replace('.xlsx','').split('__')
    inv_df=pd.read_excel(prod_path+prod_name)
    for country in new_countries:
        remapped_df=geo_remapper(inv_df, country)
        remapped_df.to_excel(path_geo_inventories+activity+'__'+country+'__'+unit+'.xlsx')
    

The following code will create new inventories where all occurences of extensive forestry are replaced with intensive forestry, and others where it's the opposite:

In [ ]:
path_forest_inventories="your_path_to_inventories_with_remapped_forestry\\"  # This is the folder where you want to save the remapped inventories with extensive and intensive flows
geo_forestry_files=os.listdir(path_forest_inventories) # This will skip files that are already in the folder
geo_inventory_files=os.listdir(path_geo_inventories) # This reads the files from the folder with remapped geographies above
# Note: This code will create a folder with inventories with only extensive and only intensive flows. To get the default case as well, you need to copy the files manually from the remapped folder into the geo_forestry folder, or modify the code
for prod_name in geo_inventory_files:
    [activity, geography, unit]=prod_name.replace('.xlsx','').split('__')
    if activity+'__'+geography+'__'+unit+'__intensive.xlsx' not in geo_forestry_files:
        inv_df=pd.read_excel(path_geo_inventories+prod_name)
        intensive_df=inv_df.copy()
        extensive_df=inv_df.copy()
        
        for i in inv_df.index:
            intensive_df.loc[i,'elementary_flow']=inv_df.loc[i,'elementary_flow'].replace('forest, extensive','forest, intensive') # Replace all extensive forestry flows with intensive forestry flows
            extensive_df.loc[i,'elementary_flow']=inv_df.loc[i,'elementary_flow'].replace('forest, intensive','forest, extensive') # Replace all intensive forestry flows with extensive forestry flows
                        
        intensive_df.to_excel(path_forest_inventories+activity+'__'+geography+'__'+unit+'__intensive.xlsx')
        extensive_df.to_excel(path_forest_inventories+activity+'__'+geography+'__'+unit+'__extensive.xlsx')    

Then we can run the impact calculations again:

In [ ]:
all_results=[]
characterized_inventories_ALL="your_path_to_a_folder_to_store_all_characterized_inventories\\"  # This is the folder where you want to save the characterized inventories for all remapped products
for prod in os.listdir(path_forest_inventories):
    # This part will create a characterized inventory and save it in the characterized_inventories folder.
    inv_df=pd.read_excel(path_forest_inventories+prod)
    characterized_inventory=mapper_land(inv_df, GLAM_land_pivot, mapping_table_GLAM, mapping_table_IW, geo_mapping, iwplus_land)
    characterized_inventory[characterized_inventory['EF_compartment'].str.contains('land')].to_excel(characterized_inventories_ALL+prod)

    # This part will gather all results in a single table:
    results=characterized_inventory[characterized_inventory['EF_compartment'].str.contains('land')][['Average Regional Impact','Average Global Impact','Marginal Regional Impact','Marginal Global Impact', 'Impact_IW+']].sum()
    results_df=pd.DataFrame(results, columns=[prod.replace('.xlsx','')])
    all_results.append(results_df)

all_results_df=pd.concat(all_results, axis=1).T